In [4]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import glob

with open("elt_cypher_queries/cleanup_nodes.cypher") as f:
    cleanup_node_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_edges.cypher") as f:
    cleanup_edge_queries = [line for line in f]

with open("elt_cypher_queries/index_nodes.cypher") as f:
    node_index_queries = [line for line in f]

node_cypher_files = glob.glob("elt_cypher_queries/nodes/*.cypher")
node_creation_queries = []
for query_file in node_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        node_creation_queries.append(creation_query)

edge_cypher_files = glob.glob("elt_cypher_queries/edges/*.cypher")
edge_creation_queries = []
for query_file in edge_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        edge_creation_queries.append(creation_query)

In [6]:
from neo4j import GraphDatabase

class IMDBELTLoader:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, params_dict={}):
        with self.driver.session(database="neo4j") as session:
            result = session.run(query, **params_dict)
     
for query in cleanup_edge_queries+cleanup_node_queries:
    print(f"Processing {query}")
    loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
    loader.execute_query(query)
    loader.close()

for query in node_creation_queries+node_index_queries+edge_creation_queries:
    print(f"Processing {query}")
    loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
    loader.execute_query(query)
    loader.close()


Processing MATCH ()-[r]->() CALL { WITH r DETACH DELETE r} IN TRANSACTIONS OF 10000 ROWS;
Processing DROP CONSTRAINT imdb_attribute_attribute IF EXISTS;

Processing DROP CONSTRAINT imdb_characters_somecharacter IF EXISTS;

Processing DROP CONSTRAINT imdb_distribution_type_distribution_type IF EXISTS;

Processing DROP CONSTRAINT imdb_genre_genre IF EXISTS;

Processing DROP CONSTRAINT imdb_job_category_category IF EXISTS;

Processing DROP CONSTRAINT imdb_language_code_alpha2 IF EXISTS;

Processing DROP CONSTRAINT imdb_name_basics_nconst IF EXISTS;

Processing DROP CONSTRAINT imdb_primaryprofession_profession IF EXISTS;

Processing DROP CONSTRAINT imdb_region_code_alpha2 IF EXISTS;

Processing DROP CONSTRAINT imdb_title_akas_id IF EXISTS;

Processing DROP CONSTRAINT imdb_title_basics_tconst IF EXISTS;

Processing DROP CONSTRAINT imdb_title_episode_epitconst IF EXISTS;

Processing DROP CONSTRAINT imdb_title_principals_id IF EXISTS;

Processing DROP CONSTRAINT imdb_title_ratings_tconst IF E

ClientError: {code: Neo.ClientError.Statement.ExternalResourceFailed} {message: Cannot load from URL 'file:///imdb_title_principal.node.csv.gz': Couldn't load the external resource at: file:///imdb_title_principal.node.csv.gz (Transactions committed: 0)}